In [1]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 10 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,736 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [2]:
!sudo apt-get update
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,016 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,521 kB]
Get:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,242 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,498 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/u

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
# Virtual display
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [2]:
import gymnasium as gym

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [3]:
# create a lunar lander environment
env = gym.make("LunarLander-v2")

# reset the environment
observation,info = env.reset()

for _ in range(20):
    # take a random action
    action = env.action_space.sample()
    print("Action : ",action)
    # do the action and it returns the next_state,reward,terminated,truncated,info
    observation,reward,terminated,truncated,info = env.step(action)
    if terminated or truncated:
        # reset the environment
        print("Environment is resetted")
        observation,info = env.reset()
env.close()

Action :  2
Action :  0
Action :  3
Action :  3
Action :  2
Action :  2
Action :  3
Action :  0
Action :  2
Action :  1
Action :  1
Action :  2
Action :  0
Action :  2
Action :  0
Action :  0
Action :  0
Action :  2
Action :  0
Action :  1


In [ ]:
# observation space
env = gym.make("LunarLander-v2")
env.reset()
print(env.observation_space.shape,env.observation_space.sample(),sep="\n")

(8,)
[61.75671    88.69034    -1.8244593   0.16196862 -1.7685957   0.81495744
  0.6944104   0.65375245]


In [ ]:
# action space
print(env.action_space.n,env.action_space.sample(),sep="\n")
env.close()

4
2


In [4]:
# model

# 1. create an environment
env = gym.make("LunarLander-v2")

model_name = "ppo-lunarlander-v2"

# 2. initialize the model
model = PPO(
    policy = "MlpPolicy", # Multiple Layer Perceptron Policy
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 5,
    gamma = 0.995,
    gae_lambda = 0.98,
    ent_coef = 0.0001,
    clip_range = 0.1,
    verbose = 1
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
# training
model.learn(total_timesteps=2500000,progress_bar=True)
model.save(model_name)

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2,500,561/2,500,000  [ 1:27:32 < 0:00:00 , 488 it/s ]

In [6]:
# evaluation done in new environment
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward,std_reward = evaluate_policy(model,eval_env,n_eval_episodes = 10 ,deterministic=True)
print(f"Mean reward : {mean_reward} +/- {std_reward}")

Mean reward : 244.81747439999998 +/- 43.787319720448856


In [7]:
# api for colab to huggingface
notebook_login()
!git config --global credential.helper store

In [8]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

model_name = "ppo-lunar-lander-v2"
model_architecture = "PPO"
env_id = "LunarLander-v2"
eval_env = DummyVecEnv([lambda: gym.make(env_id,render_mode="rgb_array")])
repo_id = f"JaiSurya/{model_architecture}-{env_id}"
commit_message = "Updating the PPO agent"

package_to_hub(
    model = model,
    model_name = model_name,
    model_architecture = model_architecture,
    env_id = env_id,
    eval_env = eval_env,
    repo_id = repo_id,
    commit_message = commit_message
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpupdjmowc/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpupdjmowc/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpupdjmowc/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpupdjmowc/-step-0-to-step-1000.mp4
ℹ Pushing repo JaiSurya/PPO-LunarLander-v2 to the Hugging Face Hub


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

policy.optimizer.pth:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

ppo-lunar-lander-v2.zip:   0%|          | 0.00/147k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/JaiSurya/PPO-LunarLander-v2/tree/main/


'https://huggingface.co/JaiSurya/PPO-LunarLander-v2/tree/main/'